In [ ]:
import operator
import random
from deap import gp, creator, base, tools

# Define primitive set
pset_routing = gp.PrimitiveSet("ROUTING", 6)  # 6 terminals cho routing rule
pset_sequencing = gp.PrimitiveSet("SEQUENCING", 6)  # 6 terminals cho sequencing

# Add operators
for pset in [pset_routing, pset_sequencing]:
    pset.addPrimitive(operator.add, 2)
    pset.addPrimitive(operator.sub, 2)
    pset.addPrimitive(operator.mul, 2)
    pset.addPrimitive(protected_div, 2)
    pset.addPrimitive(max, 2)
    pset.addPrimitive(min, 2)
    pset.addPrimitive(operator.neg, 1)
    pset.addPrimitive(abs, 1)

# Rename arguments
pset_routing.renameArguments(ARG0='req_queue_ratio')
pset_routing.renameArguments(ARG1='capacity_gap_ratio')
pset_routing.renameArguments(ARG2='median_to_req_time')
pset_routing.renameArguments(ARG3='current_to_req_time')
pset_routing.renameArguments(ARG4='demand_ratio')
pset_routing.renameArguments(ARG5='is_drone')

pset_sequencing.renameArguments(ARG0='current_to_req_time')
pset_sequencing.renameArguments(ARG1='time_since_release')
pset_sequencing.renameArguments(ARG2='time_to_close')
pset_sequencing.renameArguments(ARG3='demand_ratio')
pset_sequencing.renameArguments(ARG4='time_window_slack')
pset_sequencing.renameArguments(ARG5='release_time_ratio')

def protected_div(a, b):
    return a / b if abs(b) > 1e-6 else 1.0

In [ ]:
class FeatureCalculator:
    def __init__(self, problem):
        self.problem = problem
        self.total_demand = sum(r.demand for r in problem.requests)
        self.close_time = problem.time_horizon
    
    def compute_routing_features(self, vehicle, request, req_queue):
        """Tính 6 features cho routing rule"""
        
        # F1: Request queue ratio
        req_queue_ratio = len(req_queue) / len(self.problem.requests)
        
        # F2: Capacity gap ratio
        queue_demand = sum(r.demand for r in req_queue)
        capacity_gap = vehicle.capacity - vehicle.current_load - queue_demand
        capacity_gap_ratio = capacity_gap / self.total_demand
        
        # F3: Median to request time
        if req_queue:
            median_pos = self.compute_median_position(req_queue)
            median_to_req_time = self.distance(median_pos, request.position) / self.close_time
        else:
            median_to_req_time = 0
        
        # F4: Current to request time
        current_to_req_time = self.distance(
            vehicle.current_position, 
            request.position
        ) / self.close_time
        
        # F5: Demand ratio
        demand_ratio = request.demand / self.total_demand
        
        # F6: Vehicle type indicator
        is_drone = 1.0 if vehicle.type == 'drone' else 0.0
        
        return [
            req_queue_ratio, capacity_gap_ratio, median_to_req_time,
            current_to_req_time, demand_ratio, is_drone
        ]
    
    def compute_sequencing_features(self, vehicle, request, current_time):
        """Tính 6 features cho sequencing rule"""
        
        # F1: Current to request time
        current_to_req_time = self.distance(
            vehicle.current_position,
            request.position
        ) / self.close_time
        
        # F2: Time since release
        time_since_release = (current_time - request.release_time) / self.close_time
        
        # F3: Time to close (urgency)
        vehicle_completion_time = vehicle.finish_time_current_task()
        time_to_close = (request.close_time - vehicle_completion_time) / self.close_time
        
        # F4: Demand ratio
        demand_ratio = request.demand / self.total_demand
        
        # F5: Time window slack
        time_window_slack = (current_time - request.open_time) / self.close_time
        
        # F6: Release time ratio
        release_time_ratio = request.release_time / self.close_time
        
        return [
            current_to_req_time, time_since_release, time_to_close,
            demand_ratio, time_window_slack, release_time_ratio
        ]

In [ ]:
class DynamicVRPSimulator:
    def __init__(self, trucks, drones, requests, routing_tree, sequencing_tree):
        self.trucks = trucks
        self.drones = drones
        self.vehicles = trucks + drones
        self.requests = requests
        self.routing_func = gp.compile(routing_tree, pset_routing)
        self.sequencing_func = gp.compile(sequencing_tree, pset_sequencing)
        
        self.current_time = 0
        self.pending_requests = []
        self.served_requests = []
        self.req_queues = {v: [] for v in self.vehicles}
        
    def run_simulation(self, time_slots):
        """Chạy simulation theo các time slots"""
        
        for slot_time in time_slots:
            self.current_time = slot_time
            
            # 1. Thêm requests mới xuất hiện
            self.update_pending_requests()
            
            # 2. Routing: Assign pending requests to vehicles
            self.routing_phase()
            
            # 3. Sequencing: Sắp xếp thứ tự trong req_queue
            self.sequencing_phase()
            
            # 4. Execute: Vehicles thực hiện requests
            self.execution_phase()
        
        return self.compute_objectives()
    
    def routing_phase(self):
        """Assign requests to vehicles using routing rule"""
        
        for request in self.pending_requests[:]:
            # Tính priority score cho mỗi vehicle
            scores = []
            for vehicle in self.vehicles:
                # Check feasibility
                if not self.is_feasible(vehicle, request):
                    scores.append(float('inf'))
                    continue
                
                # Compute features
                features = self.feature_calc.compute_routing_features(
                    vehicle, request, self.req_queues[vehicle]
                )
                
                # Evaluate GP tree
                score = self.routing_func(*features)
                scores.append(score)
            
            # Assign to vehicle with minimum score
            if min(scores) < float('inf'):
                best_vehicle = self.vehicles[scores.index(min(scores))]
                self.req_queues[best_vehicle].append(request)
                self.pending_requests.remove(request)
    
    def sequencing_phase(self):
        """Sắp xếp requests trong queue của mỗi vehicle"""
        
        for vehicle in self.vehicles:
            if not self.req_queues[vehicle]:
                continue
            
            # Tính priority score cho mỗi request trong queue
            scored_requests = []
            for request in self.req_queues[vehicle]:
                features = self.feature_calc.compute_sequencing_features(
                    vehicle, request, self.current_time
                )
                score = self.sequencing_func(*features)
                scored_requests.append((score, request))
            
            # Sắp xếp theo score tăng dần (min = highest priority)
            scored_requests.sort(key=lambda x: x[0])
            self.req_queues[vehicle] = [r for _, r in scored_requests]
    
    def execution_phase(self):
        """Vehicles execute first request in their queues"""
        
        for vehicle in self.vehicles:
            if vehicle.is_available(self.current_time) and self.req_queues[vehicle]:
                request = self.req_queues[vehicle].pop(0)
                vehicle.execute_request(request, self.current_time)
                self.served_requests.append(request)
    
    def compute_objectives(self):
        """Tính 2 objectives"""
        
        # Objective 1: Makespan
        makespan = max(v.completion_time for v in self.vehicles)
        
        # Objective 2: Number served
        num_served = len(self.served_requests)
        
        # Normalize and combine (weighted sum or Pareto)
        fitness = -makespan / 1000 + num_served * 10
        
        return fitness

In [ ]:
def evolve_gp(problem, generations=100, pop_size=100):
    # Setup DEAP
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()
    
    # Generate routing and sequencing trees
    toolbox.register("routing_tree", gp.genHalfAndHalf, 
                     pset=pset_routing, min_=2, max_=5)
    toolbox.register("sequencing_tree", gp.genHalfAndHalf,
                     pset=pset_sequencing, min_=2, max_=5)
    
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.routing_tree, toolbox.sequencing_tree), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Evaluation
    def evaluate(individual):
        routing_tree, sequencing_tree = individual
        simulator = DynamicVRPSimulator(
            problem.trucks, problem.drones, problem.requests,
            routing_tree, sequencing_tree
        )
        fitness = simulator.run_simulation(problem.time_slots)
        return (fitness,)
    
    toolbox.register("evaluate", evaluate)
    toolbox.register("select", tools.selTournament, tournsize=7)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.routing_tree, pset=pset_routing)
    
    # Evolution
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", np.max)
    
    pop, log = algorithms.eaSimple(
        pop, toolbox, 
        cxpb=0.7, mutpb=0.2,
        ngen=generations,
        stats=stats, halloffame=hof,
        verbose=True
    )
    
    return hof[0], log